# 1. Setup

In [10]:
from tqdm import tqdm
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()
X = iris.data
y = np.where(iris.target == 0, 1, -1) 
print('Number of classes: %d' %len(np.unique(y)))
print('Number of data points: %d' %len(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Number of classes: 2
Number of data points: 150


In [2]:
class DecisionStump:
    def __init__(self):
        self.polarity = 1
        self.feature_index = None
        self.threshold = None
        self.alpha = None
    
    def predict(self, X):
        n_samples = X.shape[0]
        predictions = np.ones(n_samples)

        feature_column = X[:, self.feature_index]

        if self.polarity == 1:
            predictions[feature_column < self.threshold] = -1
        else:
            predictions[feature_column > self.threshold] = -1

        return predictions

In [4]:
X = np.array([[2],
              [4],
              [6],
              [8],
              [1]])

stump = DecisionStump()
stump.feature_index = 0
stump.threshold = 5
stump.polarity = 1
predictions = stump.predict(X)
print('Predictions:', predictions)

Predictions: [-1. -1.  1.  1. -1.]


In [9]:
class AdaBoost:
    def __init__(self, n_classifiers=5):
        self.n_classifiers = n_classifiers
        self.classifiers = []

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # initialize weights
        w = np.full(n_samples, (1 / n_samples))

        # start loop from t to T 
        for _ in tqdm(range(self.n_classifiers), desc="Training AdaBoost"):
            clf = DecisionStump()
            min_error = float('inf')    

            # choose feature that minimizes the weighted error
            for feature_i in range(n_features):
                X_column = X[:, feature_i]
                thresholds = np.unique(X_column)

                for threshold in thresholds:
                    predictions = np.ones(n_samples)

                    predictions[X_column < threshold] = -1

                    error = sum(w[y != predictions])

                    if error > 0.5:
                        error = 1 - error
                        p = -1
                    else:
                        p = 1

                    if error < min_error:
                        clf.polarity = p
                        clf.threshold = threshold
                        clf.feature_index = feature_i
                        min_error = error

            # calculate amount of say
            EPS = 1e-10
            clf.alpha = 0.5 * np.log((1 - min_error) / (min_error + EPS))
            predictions = clf.predict(X)

            # Update weights 
            w *= np.exp(-clf.alpha * y * predictions)
            w /= np.sum(w)
            self.classifiers.append(clf)
    
    def predict(self, X):
        clf_preds = [clf.alpha * clf.predict(X) for clf in self.classifiers]
        y_pred = np.sum(clf_preds, axis=0)
        return np.sign(y_pred)

In [11]:
model = AdaBoost(n_classifiers=5)
model.fit(X_train, y_train)

Training AdaBoost: 100%|██████████| 5/5 [00:00<00:00, 714.09it/s]


In [12]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Calculate accuracy
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

print(f"Training accuracy: {train_accuracy:.4f}")
print(f"Testing accuracy: {test_accuracy:.4f}")

Training accuracy: 0.9905
Testing accuracy: 1.0000
